<font color='blue'>
    
# ECE449 Machine Learning - Assignment 2

## Task 4: ResNet with PyTorch


[//]: # "Notebook Created by Jinghua Wang (jinghuawang@intl.zju.edu.cn), last modified on 2020-11-04"

</font>

As we design increasingly deeper NN models, it's imperative to understand how adding so many layers to our NN model can increase its complexity and expressiveness. Instead of designing and viewing by basic layers(like fully connected layers, convolution layers, and activation layers), a more handy approach is to consider the whole NN model as a composite of multiple "building blocks" (or "building block layers").


ResNet is published by He et. al. in their paper "Deep Residual Learning for Image Recognition". The paper is available online at: https://arxiv.org/abs/1512.03385. At the heart of their proposed *residual network* (*ResNet*) is the idea that every additional layer should 
more easily contain the identity function as one of its elements. 
These considerations are rather profound but they led to a surprisingly simple
solution, a *residual block*. With that, ResNet won the ImageNet Large Scale Visual Recognition Challenge in 2015. The design had a profound influence on how to build deep neural networks. Let's start exploring their idea from Residual blocks.


### Residual Blocks

Let us focus on a local part of a neural network, as depicted in the figure below. Denote the input by $\mathbf{x}$.
We assume that the desired underlying mapping we want to obtain by learning is $f(\mathbf{x})$, to be used as the input to the activation function on the top.
On the left of the figure below,
the portion within the dotted-line box 
must directly learn the mapping $f(\mathbf{x})$.
On the right,
the portion within the dotted-line box
needs to
learn the *residual mapping* $f(\mathbf{x}) - \mathbf{x}$,
which is how the residual block derives its name.
If the identity mapping $f(\mathbf{x}) = \mathbf{x}$ is the desired underlying mapping,
the residual mapping is easier to learn:
we only need to push the weights and biases
of the
upper weight layer (e.g., fully-connected layer and convolutional layer)
within the dotted-line box
to zero.
The right figure illustrates the  *residual block* of ResNet,
where the solid line carrying the layer input 
$\mathbf{x}$ to the addition operator
is called a *residual connection* (or *shortcut connection*).
With residual blocks, inputs can 
forward propagate faster through the residual connections across layers.

<img src="img/residual-block.png" width="400">



The residual block has two $3\times 3$ convolutional layers with the same number of output channels. Each convolutional layer is followed by a batch normalization layer and a ReLU activation function. Then, we skip these two convolution operations and add the input directly before the final ReLU activation function.
This kind of design requires that the output of the two convolutional layers has to be of the same shape as the input, so that they can be added together. If we want to change the number of channels, we need to introduce an additional $1\times 1$ convolutional layer to transform the input into the desired shape for the addition operation.

Your Residual nn.Module implementation should be able to generate two types of network building blocks as shown below: one where we add the input to the output before applying the ReLU nonlinearity whenever `use_1x1conv=False`, and one where we adjust channels and resolution by means of a $1 \times 1$ convolution before doing the addition operation.

<img src="img/resnet-block.png" width="500">

[//]: # "Some materials of this notebook are referred from the famous Deep Learning book 'Dive into Deep Learning' written by Aston Zhang, Zack C. Lipton, Mu Li, Alex J. Smola."

## ResNet Model

The first two layers of ResNet are the $7\times 7$ convolutional layer with 64 output channels and a stride of 2 is followed by the $3\times 3$ maximum pooling layer with a stride of 2.

ResNet uses four modules made up of residual blocks, each of which uses several residual blocks with the same number of output channels. 
The number of channels in the first module is the same as the number of input channels. Since a maximum pooling layer with a stride of 2 has already been used, it is not necessary to reduce the height and width. In the first residual block for each of the subsequent modules, the number of channels is doubled compared with that of the previous module, and the height and width are halved.

Then, we add all the modules to ResNet. Here, two residual blocks are used for each module.


There are 4 convolutional layers in each module (excluding the $1\times 1$ convolutional layer). Together with the first $7\times 7$ convolutional layer and the final fully-connected layer, there are 18 layers in total. Therefore, this model is commonly known as ResNet-18.
By configuring different numbers of channels and residual blocks in the module, we can create different ResNet models, such as the deeper 152-layer ResNet-152. The figure below contains the strcture of the full ResNet-18.

<img src="img/resnet18.png" width="200">

Now, with adequate knowledge about ResNet, let's build one and train it using PyTorch.

<font color='blue'>
    
### We provide you this file so that you can test and debug your task 4 implementations in your .py file by yourself, you don't need to modify any code in this jupyter notebook, but you are welcome to change implementations in this notebook if you want to play with this example model training process.
    
### You don't need to submit this notebook.
    
</font>

In [ ]:
# use automatic reloading for your code from task4-template.py
# remember to rename task4-template.py before you submit it.
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

from task3_template import *
from task4_template import *

### 1. Load and check your ResNet model

#### 1.1 Load and check Residual block

In [ ]:
# checking implementation without using 1x1 conv
blk = Residual(3,3)
X1 = torch.rand(4, 3, 9, 9)
print("Your block output shape is:", blk(X1).shape)
print("Here the block output shape should be: torch.Size([4, 3, 9, 9])")

In [ ]:
# checking implementation with 1x1 conv
blk = Residual(3,6, use_1x1conv=True, strides=2)
print("Your block output shape is:", blk(X1).shape)
print("Here the block output shape should be: torch.Size([4, 6, 5, 5])")

#### 1.2 Load and check the complete ResNet model

In [ ]:
net = get_resnet()

In [ ]:
X = torch.rand(size=(1, 1, 224, 224))
print("The output data shapes of all layers in your resnet are:")
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

In [ ]:
print("The output data shapes of all layers in correct lenet are:")
print(get_correct_resnet_shape_str())

#### In the two cells above, all the layer names, and the output data shapes of all the layers should be the same. 
#### (Please do not modify the layer names)

###  2. Train your ResNet model

We train ResNet on the Fashion-MNIST dataset, just like before.

Training ResNet could be time-costly, we recommend using Deep Learning accleration hardware (like CUDA compatible GPU). However, if you are using Mac or other computer without CUDA GPU, it is okay. After all, you only need to run ResNet training for 1 epoch, which could definitely be done in 60 mins.

If you are running without Deep Learning accleration hardware, we recommend that you print out test batch accuracy for every batch in each epoch in the training process, which might make things easier for debugging.

If you are using CUDA, and you encounter "CUDA output memory Runtime Error, then please use smaller batch size.

In [ ]:
lr, num_epochs, batch_size = 0.05, 1, 128
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=96)
# use the train_cnn function you have implemented in task 3 to train reset model
# We recommend not to start training before you have got the correct resnet-18 model
train_cnn(net, train_iter, test_iter, num_epochs, lr, device=try_gpu())

In [ ]:
test_dataset_accuracy = evaluate_accuracy_try_gpu(net, test_iter)
print("After model training, your ResNet-18 achieves test dataset accuracy:",test_dataset_accuracy)

#### After 1 epoch of training with learning rate 0.01-0.1, batch size 64-512, a test dataset accuracy(on all 10000 test samples) >0.7500 is usually considered correct and will give you a full score.

Congratulations! Now you have finished all the programming tasks of assignment 2!